In [110]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np
import keras.utils
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
import keras.losses

In [111]:
pd_gan =pd.read_csv("Gan_output.csv")
pd_vae =pd.read_csv("VAE_output.csv")

In [112]:
pd_gan = pd_gan.sort_values(['label'])
pd_vae = pd_vae.sort_values(['label'])

In [113]:
subset= pd_gan.loc[pd_gan['label'] == 0]
train_gan, test_gan = train_test_split(subset, test_size=0.1)
for i in range (1, 10):
    subset = pd_gan.loc[pd_gan['label'] == i]
    train, test = train_test_split(subset, test_size=0.1)
    train_gan = pd.concat([train_gan, train])
    test_gan = pd.concat([test_gan, test])
    

In [114]:
subset= pd_vae.loc[pd_vae['label'] == 0]
train_vae, test_vae = train_test_split(subset, test_size=0.1)
for i in range (1, 10):
    subset = pd_vae.loc[pd_vae['label'] == i]
    train, test = train_test_split(subset, test_size=0.1)
    train_vae = pd.concat([train_vae, train])
    test_vae = pd.concat([test_vae, test])
    


In [115]:
train_gan = train_gan.sample(frac=1).reset_index(drop=True)
x_values = train_gan.drop(['label'], axis=1)
x_train_GAN = x_values.as_matrix()
y_train_GAN = train_gan['label'].tolist()

test_gan = test_gan.sample(frac=1).reset_index(drop=True)
x_values = test_gan.drop(['label'], axis=1)
x_test_GAN = x_values.as_matrix()
y_test_GAN = test_gan['label'].tolist()

train_vae = train_vae.sample(frac=1).reset_index(drop=True)
x_values = train_vae.drop(['label'], axis=1)
x_train_VAE = x_values.as_matrix()
y_train_VAE = train_vae['label'].tolist()

test_vae = test_vae.sample(frac=1).reset_index(drop=True)
x_values = test_vae.drop(['label'], axis=1)
x_test_VAE = x_values.as_matrix()
y_test_VAE = test_vae['label'].tolist()

/home/linuxbrew/.linuxbrew/opt/python3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/linuxbrew/.linuxbrew/opt/python3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/home/linuxbrew/.linuxbrew/opt/python3/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]
/home/linuxbrew/.linuxbrew/opt/python3/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [116]:
train_vae.head()

,label,Pixel1,Pixel2,Pixel3,Pixel4,Pixel5,Pixel6,Pixel7,Pixel8,Pixel9,...,Pixel775,Pixel776,Pixel777,Pixel778,Pixel779,Pixel780,Pixel781,Pixel782,Pixel783,Pixel784
0,5,6.282638e-05,4.842842e-05,4.011890e-05,6.654814e-05,0.000131,0.000085,0.000043,0.000143,0.000036,...,0.000034,0.000041,0.000052,0.000218,0.000626,0.001325,0.001343,0.002833,1.192217e-03,7.593835e-05
1,0,6.807318e-05,5.812119e-05,2.647939e-04,1.763965e-04,0.000554,0.001014,0.004057,0.006623,0.020755,...,0.017604,0.013493,0.026134,0.014441,0.005305,0.001259,0.000342,0.000176,1.885523e-04,1.327526e-04
2,3,1.942305e-08,4.274524e-08,9.439393e-08,9.534464e-08,0.000002,0.000011,0.000028,0.000113,0.000309,...,0.419842,0.097851,0.018908,0.003993,0.001765,0.000257,0.000008,0.000001,1.443341e-07,3.537910e-08
3,8,4.411118e-05,2.022621e-05,3.839586e-05,5.154927e-05,0.000047,0.000110,0.000020,0.000088,0.000035,...,0.000149,0.000149,0.000059,0.000034,0.000024,0.000021,0.000018,0.000029,4.016070e-05,6.078504e-05
4,4,1.350680e-07,3.761456e-07,6.453870e-07,4.668091e-07,0.000015,0.000055,0.000080,0.000789,0.001556,...,0.511309,0.212991,0.086042,0.014807,0.003597,0.000589,0.000040,0.000004,1.039209e-06,3.136589e-07


In [117]:
model = Sequential()
model.add(Conv2D(8, kernel_size=3,input_shape=(28,28,1),padding='valid'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(16,kernel_size=3, padding='valid'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(32, kernel_size=3,padding='valid'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())

model.add(Dense(128))
model.add(LeakyReLU(alpha=0.1))                  
model.add(Dense(10, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 26, 26, 8)         80        
_________________________________________________________________
leaky_re_lu_21 (LeakyReLU)   (None, 26, 26, 8)         0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 13, 13, 8)         0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 11, 11, 16)        1168      
_________________________________________________________________
leaky_re_lu_22 (LeakyReLU)   (None, 11, 11, 16)        0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 5, 5, 16)          0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 3, 3, 32)          4640      
__________

In [118]:
def do_classification(x_train, x_test, y_train, y_test, batch_size, epochs, num_classes):

    x_train = x_train.reshape(-1, 28,28, 1)
    x_test = x_test.reshape(-1, 28,28, 1)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    y_train_one_hot = keras.utils.to_categorical(y_train)
    y_test_one_hot = keras.utils.to_categorical(y_test)

    x_train,valid_X,train_label,valid_label = train_test_split(x_train, y_train_one_hot,test_size=0.2)

    model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

    trained = model.fit(x_train, train_label, batch_size=batch_size, epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))


    test_eval = model.predict(x_test)
    return test_eval,y_test

In [119]:
batch_size = 64
epochs= 12
num_classes = 10
GAN_predicted, GAN_actual = do_classification(x_train = x_train_GAN, x_test = x_test_GAN, y_train = y_train_GAN, y_test = y_test_GAN, batch_size = batch_size, epochs = epochs, num_classes= num_classes)

Train on 7200 samples, validate on 1800 samples
Epoch 1/12
7200/7200 [==============================] - 3s 478us/step - loss: 1.1206 - acc: 0.6644 - val_loss: 0.3548 - val_acc: 0.8583
Epoch 2/12
7200/7200 [==============================] - 3s 371us/step - loss: 0.2288 - acc: 0.9107 - val_loss: 0.1835 - val_acc: 0.9233
Epoch 3/12
7200/7200 [==============================] - 3s 374us/step - loss: 0.1457 - acc: 0.9410 - val_loss: 0.1182 - val_acc: 0.9539
Epoch 4/12
7200/7200 [==============================] - 3s 375us/step - loss: 0.1009 - acc: 0.9618 - val_loss: 0.0759 - val_acc: 0.9800
Epoch 5/12
7200/7200 [==============================] - 3s 372us/step - loss: 0.0642 - acc: 0.9815 - val_loss: 0.0353 - val_acc: 0.9939
Epoch 6/12
7200/7200 [==============================] - 3s 373us/step - loss: 0.0301 - acc: 0.9933 - val_loss: 0.0251 - val_acc: 0.9961
Epoch 7/12
7200/7200 [==============================] - 3s 374us/step - loss: 0.0256 - acc: 0.9935 - val_loss: 0.0172 - val_acc: 0.9978


In [120]:
batch_size = 64
epochs= 50
num_classes = 10
VAE_predicted, VAE_actual = do_classification(x_train = x_train_VAE, x_test = x_test_VAE, y_train = y_train_VAE, y_test = y_test_VAE, batch_size = batch_size, epochs = epochs, num_classes= num_classes)

Train on 7200 samples, validate on 1800 samples
Epoch 1/50
7200/7200 [==============================] - 3s 471us/step - loss: 1.2346 - acc: 0.7065 - val_loss: 0.6989 - val_acc: 0.7567
Epoch 2/50
7200/7200 [==============================] - 3s 369us/step - loss: 0.7228 - acc: 0.7439 - val_loss: 0.6241 - val_acc: 0.7767
Epoch 3/50
7200/7200 [==============================] - 3s 369us/step - loss: 0.6633 - acc: 0.7578 - val_loss: 0.6163 - val_acc: 0.7717
Epoch 4/50
7200/7200 [==============================] - 3s 365us/step - loss: 0.6364 - acc: 0.7650 - val_loss: 0.5844 - val_acc: 0.7833
Epoch 5/50
7200/7200 [==============================] - 3s 365us/step - loss: 0.6142 - acc: 0.7703 - val_loss: 0.5715 - val_acc: 0.7894
Epoch 6/50
7200/7200 [==============================] - 3s 365us/step - loss: 0.5972 - acc: 0.7821 - val_loss: 0.6142 - val_acc: 0.7611
Epoch 7/50
7200/7200 [==============================] - 3s 367us/step - loss: 0.5789 - acc: 0.7869 - val_loss: 0.5326 - val_acc: 0.8072


In [121]:
VAE_predicted_y = [np.argmax(i) for i in VAE_predicted]
GAN_predicted_y = [np.argmax(i) for i in GAN_predicted]

In [127]:
y_pred = pd.Series(GAN_predicted_y, name='GAN Predicted')
y_actu = pd.Series(GAN_actual, name='GAN Actual')
df_confusion_GAN = pd.crosstab(y_actu, y_pred)
df_confusion_GAN

GAN Predicted,0,1,2,3,4,5,6,7,8,9
GAN Actual,,,,,,,,,,
0,99,0,0,1,0,0,0,0,0,0
1,0,100,0,0,0,0,0,0,0,0
2,0,0,100,0,0,0,0,0,0,0
3,0,0,0,99,1,0,0,0,0,0
4,0,0,0,0,100,0,0,0,0,0
5,0,0,0,0,0,100,0,0,0,0
6,0,0,0,0,0,0,100,0,0,0
7,0,0,0,0,0,0,0,100,0,0
8,0,0,0,0,0,0,0,0,100,0


In [128]:
y_pred = pd.Series(VAE_predicted_y, name='VAE Predicted')
y_actu = pd.Series(VAE_actual, name='VAE Actual')
df_confusion_VAE = pd.crosstab(y_actu, y_pred)
df_confusion_VAE

VAE Predicted,0,1,2,3,4,5,6,7,8,9
VAE Actual,,,,,,,,,,
0,81,1,5,2,0,0,10,0,1,0
1,0,97,2,1,0,0,0,0,0,0
2,2,0,83,2,4,0,9,0,0,0
3,4,3,3,82,2,1,5,0,0,0
4,0,0,9,3,80,0,8,0,0,0
5,0,0,0,1,0,86,1,5,1,6
6,10,1,19,3,9,0,58,0,0,0
7,0,0,0,0,0,1,0,88,0,11
8,0,0,1,0,0,1,2,0,96,0
